In [1]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd

from typing import Tuple, List, Dict
import tensorflow.keras as K
from tensorflow.keras.preprocessing.sequence import pad_sequences, TimeseriesGenerator
from sklearn.preprocessing import OneHotEncoder

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.listdir('../resources')

['report_images',
 '.DS_Store',
 'cityu_msr_20.json',
 '.gitkeep',
 'models',
 'pku_30.json',
 'cityu_20.json',
 'logging']

In [3]:
loaded_model = K.models.load_model('../resources/models/model_2019-04-23_16:18:12_-0500.h5')
loaded_model.load_weights('../resources/models/model_weights_2019-04-23_16:18:12_-0500.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
loaded_model.compile(loss = 'categorical_crossentropy',
              optimizer = K.optimizers.Adam(lr=0.0015, clipnorm=1., clipvalue=0.5),
              metrics = ['acc', K.metrics.Precision()])

In [5]:
from createdataset import *

In [ ]:
score = loaded_model.evaluate(data["dev"]["X"], data["dev"]["y"], verbose=3)
print("Test data: loss = {:.1f}".format(score[0]))
for i in range(1,3):
    print("%s: %.2f%%" % (loaded_model.metrics_names[i], score[i]*100))

# predict .py

In [6]:
from createdataset import *
import json

In [7]:
Label_file, Input_file = ChooseDataset('dev', ['cityu'])
Input_file

'../icwb2-data/gold/cityu_test_gold_simplified_InputFile.utf8'

In [8]:
with open('../resources/pku_30.json') as f:
    loaded_data = json.load(f)

A = CreateDataset(None, Input_file, None, 'TEST', [loaded_data['UNIGRAM_DICT'],
                                                   loaded_data['BIGRAM_DICT']])
X_test_uni, X_test_bi = A.DateGenTest()
TEST_DATA = {"X": [X_test_uni, X_test_bi]}


In [11]:
del X_test_uni[-1]
del X_test_bi[-1]

In [34]:
#e = loaded_model.predict(X_test_bi)

In [100]:
padded = lambda x: np.array([xi+[1]*(max(map(len, x))-len(xi)) for xi in x])
lengths = [len(i) for i in X_test_uni]
#pad to max line length
q = loaded_model.predict([padded(X_test_uni), padded(X_test_bi)])
#de - one hot encode
q = np.argmax(q, axis=2)

In [103]:
to_bies = []
for i in range(len(q)):
    to_bies.append(list(q[i][:lengths[i]]))


In [115]:
to_bies

[[3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 [3,
  3,
  3,
  3,
  3,
  0,
  1,
  2,
  3,
  0,
  2,
  3,
  0,
  1,
  2,
  3,
  3,
  3,
  3,
  3,
  0,
  2,
  3,
  0,
  2,
  3,
  3,
  3,
  0,
  2,
  3,
  3,
  3,
  3,
  3,
  0,
  3,
  3,
  3,
  3,
  0,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  2,
  3,
  3,
  3,
  3,
  3,
  0,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  2,
  3,
  3,
  3,
  3],
 [2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  2,
  3,
  3,
  3

In [97]:
q[:,:,:].shape

(1492, 165, 4)

We only have the input file and the dictionary

# need to save to file when training model
1. Dictonaries [uni_word_to_idx, bi_word_to_idx]
2. padding size

In [ ]:
to_array = lambda x: np.array([np.array(xi) for xi in x])

In [ ]:
X_test_uni = to_array(X_test_uni)
X_test_bi = to_array(X_test_bi)

In [ ]:
X_test_uni[-1]

In [ ]:
len(X_test_uni[0])

In [ ]:
len(X_test_bi[0])

[]

In [17]:
SUBSET = ['pku']
PADDING_SIZE = 30
data, uni_word_to_idx, bi_word_to_idx = data_feed(SUBSET, PADDING_SIZE)


*****training*****
X uni-bi shape: (19056, 30)(19056, 30)
y shape: (19056, 30, 4)
*****dev*****
X uni-bi shape: (1945, 30)(1945, 30)
y shape: (1945, 30, 4)


In [21]:
x =loaded_model.predict(data['dev']['X'])

In [23]:
x.shape

(1945, 30, 4)

In [32]:
data['train']['X']

[array([[2854, 1565, 2238, ...,    0,    0,    0],
        [2160, 3317, 2160, ...,    0,    0,    0],
        [2757,    9, 3596, ...,    0,    0,    0],
        ...,
        [2929, 2689, 2153, ...,    0,    0,    0],
        [1367, 2708, 2011, ...,    0,    0,    0],
        [ 943,  756, 1362, ...,    0,    0,    0]], dtype=int32),
 array([[133508, 148694,  73885, ...,      0,      0,      0],
        [ 19731, 134876, 271262, ...,      0,      0,      0],
        [146778,  66135, 187844, ...,      0,      0,      0],
        ...,
        [ 70112,  70897, 204903, ...,      0,      0,      0],
        [184114,  76834, 103969, ...,      0,      0,      0],
        [247197, 182495, 112797, ...,      0,      0,      0]], dtype=int32)]